# Basic LSTM usage example using MNIST
### Sungjoon Choi (sungjoon.choi@cpslab.snu.ac.kr)
#### based on [Damien's example](https://github.com/aymericdamien)

In [1]:
# Import packages
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
print ("Packages imported")

# Load MNIST, our beloved friend
mnist = input_data.read_data_sets("data/", one_hot=True)
trainimgs, trainlabels, testimgs, testlabels \
 = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels 
ntrain, ntest, dim, nclasses \
 = trainimgs.shape[0], testimgs.shape[0], trainimgs.shape[1], trainlabels.shape[1]
print ("MNIST loaded")

Packages imported
Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
MNIST loaded


### We will treat the MNIST image $\in \mathcal{R}^{28 \times 28}$ as $28$ sequences of a vector $\mathbf{x} \in \mathcal{R}^{28}$. 
### Our simple RNN consists of  
1. One input layer which converts a $28$ dimensional input to an $128$ dimensional hidden layer, 
2. one intermediate recurrent neural network (LSTM) 
3. and one output layer which converts an $128$ dimensional output of the LSTM to $10$ dimensional output indicating a class label. 

<img src="images/etc/rnn_input3.jpg" width="700" height="400" >

# Contruct Network

In [2]:
# Recurrent neural network 
diminput  = 28
dimhidden = 128
dimoutput = nclasses
nsteps    = 28
weights = {
    'hidden': tf.Variable(tf.random_normal([diminput, dimhidden])), 
    'out': tf.Variable(tf.random_normal([dimhidden, dimoutput]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([dimhidden])),
    'out': tf.Variable(tf.random_normal([dimoutput]))
}
def _RNN(_X, _istate, _W, _b, _nsteps, _name):
    # 1. Permute input from [batchsize, nsteps, diminput] => [nsteps, batchsize, diminput]
    _X = tf.transpose(_X, [1, 0, 2])
    # 2. Reshape input to [nsteps*batchsize, diminput] 
    _X = tf.reshape(_X, [-1, diminput])
    # 3. Input layer => Hidden layer
    _H = tf.matmul(_X, _W['hidden']) + _b['hidden']
    # 4. Splite data to 'nsteps' chunks. An i-th chunck indicates i-th batch data 
    _Hsplit = tf.split(0, _nsteps, _H) 
    # 5. Get LSTM's final output (_O) and state (_S)
    #    Both _O and _S consist of 'batchsize' elements
    with tf.variable_scope(_name):
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(dimhidden, forget_bias=1.0)
        _LSTM_O, _LSTM_S = tf.nn.rnn(lstm_cell, _Hsplit, initial_state=_istate)
    # 6. Output
    _O = tf.matmul(_LSTM_O[-1], _W['out']) + _b['out']    
    # Return! 
    return {
        'X': _X, 'H': _H, 'Hsplit': _Hsplit,
        'LSTM_O': _LSTM_O, 'LSTM_S': _LSTM_S, 'O': _O 
    }
print ("Network ready")

Network ready


# Out Network looks like this

<img src="images/etc/rnn_mnist_look.jpg" width="700" height="400" >

# Define functions

In [3]:
learning_rate = 0.001
x      = tf.placeholder("float", [None, nsteps, diminput])
istate = tf.placeholder("float", [None, 2*dimhidden]) #state & cell => 2x n_hidden
y      = tf.placeholder("float", [None, dimoutput])
myrnn  = _RNN(x, istate, weights, biases, nsteps, 'basic')
pred   = myrnn['O']
cost   = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) 
optm   = tf.train.AdamOptimizer(learning_rate).minimize(cost) # Adam Optimizer
accr   = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred,1), tf.argmax(y,1)), tf.float32))
init   = tf.initialize_all_variables()
print ("Network Ready!")

Network Ready!


# Run!

In [4]:
training_epochs = 5
batch_size      = 128
display_step    = 1
sess = tf.Session()
sess.run(init)
summary_writer = tf.train.SummaryWriter('/tmp/tensorflow_logs', graph=sess.graph)
print ("Start optimization")
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape((batch_size, nsteps, diminput))
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys 
                                              , istate: np.zeros((batch_size, 2*dimhidden))})/total_batch
    # Display logs per epoch step
    if epoch % display_step == 0: 
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys, istate: np.zeros((batch_size, 2*dimhidden))})
        print (" Training accuracy: %.3f" % (train_acc))
        testimgs = testimgs.reshape((ntest, nsteps, diminput))
        test_acc = sess.run(accr, feed_dict={x: testimgs, y: testlabels, istate: np.zeros((ntest, 2*dimhidden))})
        print (" Test accuracy: %.3f" % (test_acc))
print ("Optimization Finished.")

Start optimization
Epoch: 000/005 cost: 0.565064912
 Training accuracy: 0.977
 Test accuracy: 0.876
Epoch: 001/005 cost: 0.154534578
 Training accuracy: 0.969
 Test accuracy: 0.953
Epoch: 002/005 cost: 0.097276793
 Training accuracy: 0.969
 Test accuracy: 0.963
Epoch: 003/005 cost: 0.076267744
 Training accuracy: 0.977
 Test accuracy: 0.969
Epoch: 004/005 cost: 0.057375757
 Training accuracy: 1.000
 Test accuracy: 0.977
Optimization Finished.


# What we have done so far is to feed 28 sequences of vectors $ \mathbf{x} \in \mathcal{R}^{28}$. 

# What will happen if we only feed 14 sequences of $\mathbf{x}$?

In [5]:
# How may sequences will we use?
nsteps2     = 25

# Test
testimgs = testimgs.reshape((ntest, nsteps, diminput))
testimgs_trucated = np.zeros(testimgs.shape)
testimgs_trucated[:, 28-nsteps2:] = testimgs[:, :nsteps2, :]

test_acc = sess.run(accr, feed_dict={x: testimgs_trucated, y: testlabels, istate: np.zeros((ntest, 2*dimhidden))})
print (" If we use %d seqs, test accuracy becomes %.3f" % (nsteps2, test_acc))

 If we use 25 seqs, test accuracy becomes 0.772


# What's going on inside the RNN?

In [6]:
batch_size = 5
# Get minibatch
xtest, _ = mnist.test.next_batch(batch_size)
# Reshape (this will go into the network)
xtest1 = xtest.reshape((batch_size, nsteps, diminput))
rnnout_X = sess.run(myrnn['X'], feed_dict={x: xtest1, istate: np.zeros((batch_size, 2*dimhidden))})
rnnout_H = sess.run(myrnn['H'], feed_dict={x: xtest1, istate: np.zeros((batch_size, 2*dimhidden))})
rnnout_Hsplit = sess.run(myrnn['Hsplit'], feed_dict={x: xtest1, istate: np.zeros((batch_size, 2*dimhidden))})
rnnout_LSTM_O = sess.run(myrnn['LSTM_O'], feed_dict={x: xtest1, istate: np.zeros((batch_size, 2*dimhidden))})
rnnout_O = sess.run(myrnn['O'], feed_dict={x: xtest1, istate: np.zeros((batch_size, 2*dimhidden))})

print ("Shape of 'xtest' is %s" % (xtest.shape,))
print ("Shape of 'xtest1' is %s" % (xtest1.shape,))
print ("Shape of 'rnnout_X' is %s" % (rnnout_X.shape,))
print ("Shape of 'rnnout_H' is %s" % (rnnout_H.shape,))
print ("Length of 'rnnout_Hsplit' is %s and the shape of each item is %s" 
       % (len(rnnout_Hsplit), rnnout_Hsplit[0].shape))
print ("Length of 'rnnout_LSTM_O' is %s and the shape of each item is %s" 
       % (len(rnnout_LSTM_O), rnnout_LSTM_O[0].shape))
print ("Shape of 'rnnout_O' is %s" % (rnnout_O.shape,))


Shape of 'xtest' is (5, 784)
Shape of 'xtest1' is (5, 28, 28)
Shape of 'rnnout_X' is (140, 28)
Shape of 'rnnout_H' is (140, 128)
Length of 'rnnout_Hsplit' is 28 and the shape of each item is (5, 128)
Length of 'rnnout_LSTM_O' is 28 and the shape of each item is (5, 128)
Shape of 'rnnout_O' is (5, 10)
